In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
from glob import glob

In [43]:
import os
from typing import List, Dict

def generate_manual_index(root_dir: str, manual_type: str = "user") -> List[Dict]:
    """
    지정된 루트 디렉토리에서 하위 메뉴얼 디렉토리를 순회하며,
    .md 파일과 img 폴더의 이미지들을 분석하여 지정된 JSON 형태로 반환합니다.

    Args:
        root_dir (str): 예: './firstUser'
        manual_type (str): 예: 'user', 'admin', 'firstUser'

    Returns:
        List[Dict]: 메뉴얼 JSON 리스트
    """
    result = []
    base_dir_name = os.path.basename(os.path.normpath(root_dir))

    for entry in os.scandir(root_dir):
        if entry.is_dir():
            manu_type = entry.name
            md_path = os.path.join(entry.path, f"{manu_type}.md")
            img_dir = os.path.join(entry.path, "img")

            if not os.path.isfile(md_path):
                continue  # .md가 없으면 스킵

            # 이미지 URL 리스트 구성
            image_urls = []
            if os.path.isdir(img_dir):
                for img_file in sorted(os.listdir(img_dir)):
                    if img_file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
                        image_urls.append(
                            f"https://doc.tg-cloud.co.kr/manual/console/{base_dir_name}/{manu_type}/img/{img_file}"
                        )

            # 최종 JSON 구조 생성
            result.append({
                "text": f"{manu_type}.md",
                "metadata": {
                    "manual_type": manual_type,
                    "manu_type": manu_type,
                    "source_url": f"https://doc.tg-cloud.co.kr/manual/console/{base_dir_name}/{manu_type}/{manu_type}",
                    "image_url": image_urls
                }
            })

    return result


In [44]:
import os
import re
from typing import List, Dict

def generate_chunks_from_index(index_list: List[Dict], base_dir: str) -> List[Dict]:
    """
    Markdown 파일을 직접 줄 단위로 읽고 섹션(##) 기준으로 나누어 chunk 생성
    - 목차 섹션은 제외
    - ![](img/xxx.png) 패턴에서 이미지 파일명 추출하여 image_url 매핑
    """
    all_chunks = []

    for item in index_list:
        md_filename = item["text"]
        manu_type = item["metadata"]["manu_type"]
        manual_type = item["metadata"]["manual_type"]
        source_url = item["metadata"]["source_url"]
        image_url_list = item["metadata"]["image_url"]

        md_path = os.path.join(base_dir, manu_type, md_filename)
        if not os.path.isfile(md_path):
            print(f"⚠️ 파일 없음: {md_path}")
            continue

        with open(md_path, "r", encoding="utf-8") as f:
            lines = f.readlines()

        current_section = "__intro__"
        current_text_lines = []
        skip_section = False

        def resolve_image_urls(text: str) -> List[str]:
            matches = re.findall(r'!\[.*?\]\((.*?)\)', text)
            urls = []
            for filename in matches:
                for full_url in image_url_list:
                    if filename in full_url:
                        urls.append(full_url)
            return urls

        for line in lines:
            # 섹션 시작
            if line.strip().startswith("## "):
                # 이전 섹션 저장
                if current_text_lines and not skip_section:
                    full_text = "".join(current_text_lines).strip()
                    if full_text:
                        chunk = {
                            "text": full_text,
                            "metadata": {
                                "manual_type": manual_type,
                                "manu_type": manu_type,
                                "section": current_section,
                                "source_url": source_url,
                                "image_url": resolve_image_urls(full_text)
                            }
                        }
                        all_chunks.append(chunk)

                current_section = line.strip().replace("##", "").strip()
                skip_section = (current_section == "목차")
                current_text_lines = []
            else:
                if not skip_section:
                    current_text_lines.append(line)

        # 마지막 섹션 저장
        if current_text_lines and not skip_section:
            full_text = "".join(current_text_lines).strip()
            if full_text:
                chunk = {
                    "text": full_text,
                    "metadata": {
                        "manual_type": manual_type,
                        "manu_type": manu_type,
                        "section": current_section,
                        "source_url": source_url,
                        "image_url": resolve_image_urls(full_text)
                    }
                }
                all_chunks.append(chunk)

    return all_chunks


In [45]:
# 1. 메뉴얼 인덱스 생성
manual_index = generate_manual_index("./manual/user/firstUser", manual_type="firstUser")

# 2. 청크 생성
chunks = generate_chunks_from_index(manual_index, base_dir="./manual/user/firstUser")

# 3. 결과 확인
print(f"총 {len(chunks)}개 청크 생성됨")
# for chunk in chunks:
#     display(json.dumps(chunk, indent=2, ensure_ascii=False))
#     print("-" * 80)  # 구분선 추가

import pandas as pd
# 열(column) 너비 제한 해제
pd.set_option('display.max_colwidth', None)

# (선택) 출력 행 개수나 전체 열 개수도 확장 가능
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

df = pd.json_normalize(chunks)
df

총 16개 청크 생성됨


,text,metadata.manual_type,metadata.manu_type,metadata.section,metadata.source_url,metadata.image_url
0,"# Namespaces\n\n> Namespaces는 상단의 클러스터에서 서비스중인 Namespace 목록을 확인하고 생성, 삭제하는 서비스 입니다.&#x20;\n\n---",firstUser,namespaces,__intro__,https://doc.tg-cloud.co.kr/manual/console/firstUser/namespaces/namespaces,[]
1,1. [Namespace 메뉴 진입](#1-namespace-메뉴-진입)\n2. [namespace 생성 신청](#2-namespace-생성-신청)\n3. [namespace 생성 확인](#3-namespace-생성-확인),firstUser,namespaces,**목차**,https://doc.tg-cloud.co.kr/manual/console/firstUser/namespaces/namespaces,[]
2,1. 좌측 메뉴 `Namespaces` 메뉴 클릭\n\n ![](img/namespace_menu.png)\n\n - 좌측 메뉴 `Namespaces` 메뉴 클릭 후 네임스페이스 화면에 진입합니다.\n - 신규 프로젝트에 할당된 유저이며 프로젝트 하위에 속한 네임스페이스가 없기 때문에 빈 리스트로 표현됩니다.\n\n---,firstUser,namespaces,1. Namespace 메뉴 진입,https://doc.tg-cloud.co.kr/manual/console/firstUser/namespaces/namespaces,[https://doc.tg-cloud.co.kr/manual/console/firstUser/namespaces/img/namespace_menu.png]
3,"1. `생성` 버튼을 클릭하면 네임스페이스 신청을 할 수 있습니다. 네임스페이스 생성은 바로 생성되는것이 아닌 해당 `생성` 버튼을 통하여 신청 후 운영자/프로젝트 관리자의 승인을 거친 후에 네임스페이스를 생성할 수 있습니다.\n - 네임스페이스 신청 후 생성까지의 결재라인은 아래와 같이 이루어 집니다.\n\n > 유저 네임스페이스 신청 > portal 운영자 1차 결재 승인 > 프로젝트 관리자 2차 결재 승인\n \n - 아래와 같이 `생성` 버튼을 클릭하게 되면 Namespace 신청 페이지로 이동합니다. 라는 안내 문구와 함께 확인 버튼을 통하여 신청 페이지로 이동할 수 있습니다.\n\n ![](img/namespace_create_button.png)\n\n2. 네임스페이스 신청은 아래 form을 작성 후 신청할 수 있으며 항목 선택 및 입력 후 확인 버튼을 클릭하면 네임스페이스 생성 신청이 완료 됩니다. 주요 선택 항목에 대한 설명은 아래와 같습니다.\n - Project: 네임스페이스가 속하게 될 프로젝트를 선택하는 항목입니다. \n - Cluster: 프로젝트 하위에 소속된 클러스터의 목록이며 실제 신청하는 네임스페이스가 배포되는 클러스터 입니다.\n - Namespace: 신청하는 네임스페이스의 이름입니다. 검색 버튼을 통하여 클러스터 하위에 동일한 이름의 네임스페이스가 존재하는지 여부를 체크합니다.\n - 사용목적: 네임스페이스가 사용되는 목적을 작성하는 항목입니다.\n - CPU: 네임스페이스에 할당할 CPU 용량입니다.\n - Memory(GiB): 네임스페이스에 할당할 메모리 용량입니다.\n - Storage(GiB): 네임스페이스에 할당할 스토리지 용량입니다.\n\n ![](img/namespace_application.png)\n\n3. 신청을 완료하면 신청관리 화면으로 이동하게 되며 내가 신청한 네임스페이스의 목록 및 결재 단계를 확인할 수 있습니다. 운영자 > 프로젝트 관리자의 1,2차 승인 후 네임스페이스가 추가되는것을 확인할 수 있습니다.\n\n ![](img/namespace_application_management.png)\n\n---",firstUser,namespaces,2. Namespace 생성 신청,https://doc.tg-cloud.co.kr/manual/console/firstUser/namespaces/namespaces,"[https://doc.tg-cloud.co.kr/manual/console/firstUser/namespaces/img/namespace_create_button.png, https://doc.tg-cloud.co.kr/manual/console/firstUser/namespaces/img/namespace_application.png, https://doc.tg-cloud.co.kr/manual/console/firstUser/namespaces/img/namespace_application_management.png]"
4,"1. 신청관리 화면에서 운영자, 프로젝트 관리자의 1,2차 승인이 완료되면 네임스페이스가 생성이 되며 해당 화면에서 확인할 수 있습니다.\n - 직접 생성한 프로젝트 및 하위 클러스터, 승인 완료된 네임스페이스를 상단에서 선택을 한 후 네임스페이스 메뉴에 진입하면 생성된 네임스페이스를 확인할 수 있습니다.\n - 직접 신청하여 생성된 네임스페이스내에서의 부여된 권한은 네임스페이스 관리자 권한을 갖게 됩니다.\n - 네임스페이스 생성 확인 및 특정 프로젝트 > 네임스페이스 하위에 유저의 권한이 할당되면 오픈된 메뉴에 한해서 각각의 기능을 수행할 수 있습니다. (<U>**각 수행 가능한 기능은 일반 사용자 매뉴얼 참조**</U>) \n\n ![](img/namespace_create_result.png)",firstUser,namespaces,3. Namespace 생성 확인,https://doc.tg-cloud.co.kr/manual/console/firstUser/namespaces/namespaces,[https://doc.tg-cloud.co.kr/manual/console/firstUser/namespaces/img/namespace_create_result.png]
5,"# Project\n\n> 사용자에게 할당된 Project와 하위의 Namespace, 사용자 정보를 확인 할 수 있는 메뉴입니다. <br />\n> 새로운 Project를 생성 및 삭제 할 수 있고, Project 관리자나 사용자를 추가 및 삭제 할 수 있습니다.\n\n---",firstUser,project,__intro__,https://doc.tg-cloud.co.kr/manual/console/firstUser/project/project,[]
6,1. 좌측 메뉴 `Project` 메뉴 클릭\n\n\n ![](img/project_menu.png)\n\n\n - 좌측 메뉴 `Project` 메뉴 클릭 후 프로젝트 화면에 진입합니다.\n - 최초 로그인한 사용자에게 할당된 프로젝트가 없어 빈 리스트로 표현됩니다.\n\n---,firstUser,project,1. Project 메뉴 확인,https://doc.tg-cloud.co.kr/manual/console/firstUser/project/project,[https://doc.tg-cloud.co.kr/manual/console/firstUser/project/img/project_menu.png]
7,1. `생성` 버튼을 클릭하여 프로젝트를 생성할 수 있습니다.\n\n![](img/project_create_button.png)\n\n2. 프로젝트 이름만 입력 후 생성할 수 있습니다. 있으며 \n\n![](img/project_create.png)\n\n---,firstUser,project,2. Project 생성,https://doc.tg-cloud.co.kr/manual/console/firstUser/project/project,"[https://doc.tg-cloud.co.kr/manual/console/firstUser/project/img/project_create_button.png, https://doc.tg-cloud.co.kr/manual